In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

Make Initial Dataframe

In [ ]:
whatsapp_df = pd.read_fwf('_chat.txt', header = None)

whatsapp_df

Remove Secondary Lines of Mulitline Code (Temporary Solution)

In [ ]:
whatsapp_df = whatsapp_df[whatsapp_df[0].str[0] == '[']
whatsapp_df

Reorganize Columns to contain the Message, Date and Time on Different Columns while Removing the Other Columns

In [ ]:
whatsapp_df[['Datetime', 'Message']] = whatsapp_df[0].str.split('] ', n = 1, expand = True)
whatsapp_df['Datetime'] = whatsapp_df['Datetime'].str[1:]
whatsapp_df[['Date', 'Time']] = whatsapp_df['Datetime'].str.split(', ', n = 1, expand = True)

whatsapp_df.drop(columns=[col for col in whatsapp_df if col not in ['Message', 'Date', 'Time']], inplace=True)
whatsapp_df

Add User Column, Removing All Messages Not Sent By A User

In [ ]:
whatsapp_df = whatsapp_df[whatsapp_df['Message'].str.contains(':') == True]
whatsapp_df[['User', 'Message']] = whatsapp_df['Message'].str.split(': ', n = 1, expand = True)
whatsapp_df

Extract Group Chat Name

In [ ]:
group_chat_name = whatsapp_df.iloc[0]['User'] # This is the message WhatsApp sends about end to end encryption
whatsapp_df = whatsapp_df.iloc[1:,:] # Removes the message WhatsApp sends about end to end encryption
group_chat_name

Remove Media

In [ ]:
whatsapp_df = whatsapp_df[whatsapp_df['Date'].str[0] != '[']
whatsapp_df

Remove Calls

In [ ]:
whatsapp_df = whatsapp_df[~(whatsapp_df['Message'].str.contains(' started a video call') | whatsapp_df['Message'].str.contains(' started a call'))]
whatsapp_df

Find the User Who Sent the Most Messages

In [ ]:
user_df = whatsapp_df.groupby('User', as_index = False).count().sort_values(by = 'Message')
user_df.drop(columns = ['Date', 'Time'], inplace = True)
user_df = user_df.rename(columns = {'User': 'User', 'Message': 'Messages Sent'})
user_df['Percentages'] = np.round(user_df['Messages Sent'].divide(other = (whatsapp_df.shape[0]/100)), 1)
user_df

Plot This Data

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
fig.tight_layout(pad=3.0)
ax.pie(user_df['Percentages'], labels=['' for x in range(len(user_df))], autopct='%1.1f%%',
        shadow=True, startangle=90)
ax.axis('equal')
ax.set_title(f'Percentage of Messages Sent in \'{group_chat_name}\'')
plt.legend(loc="upper left", labels = user_df['User'] + ' (' + user_df['Percentages'].astype(str) + '%)'  )
plt.show()